In [12]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import tqdm
import random

from min_max_agent import MinMaxAgent
from qlearning_agent import QleaningAgent, play_game
from tic_tac_toe import TicTacToe, TikTakCounter
from mcts_agent_v1 import MCTSAgent
from mcts_agent import MCTSAgent as MC

Крестики-нолики -- это, конечно, далеко не го, и обычный альфа-бета поиск с отсечением здесь наверняка может работать идеально вплоть до довольно больших досок. Однако мы всё-таки для этого учебного задания будем реализовывать более практически релевантный метод MCTS -- заодно фактически получится и упражнение на многоруких бандитов.

Вспомогательные функции

In [5]:
def random_step(env: TicTacToe):
    actions = env.getEmptySpaces()
    return random.choice(actions)


def update_counter(counter: TikTakCounter, reward):
    if reward == 1:
        counter.cross += 1
    elif reward == -1:
        counter.naughts += 1
    elif reward == 0:
        counter.draw += 1
    else:
        counter.invalid += 1
    counter.tot += 1


def play_rand_game(agent, env, counter: TikTakCounter):
    done = False

    while not done:
        action = agent.get_action(env)
        _, reward, done, _ = env.step(action)
        if done:
            update_counter(counter, reward)
            break
        action = random_step(env)
        _, reward, done, _ = env.step(action)
        if done:
            update_counter(counter, reward)

Сыграем в игру со случайным противником

In [3]:
%pdb

Automatic pdb calling has been turned ON


In [10]:
mtcs_agent = MCTSAgent(4,4,4,num_rounds=300_000)
mtcs_agent.train()

100%|██████████| 300000/300000 [05:13<00:00, 955.86it/s] 


In [13]:

random.seed(1000)
counter = TikTakCounter()
env = TicTacToe(4, 4, 4)
acent = MC(num_rounds=1000)
for i in tqdm.tqdm_notebook(range(100)):
    env.reset()
    play_rand_game(acent, env, counter)
    if i % 10 == 0:
        print(counter)

<ipython-input-13-283cc518289e>:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm.tqdm_notebook(range(100)):


cross=0 naughts=0 tot=1 draw=1 invalid=0
cross=2 naughts=0 tot=11 draw=9 invalid=0
cross=4 naughts=2 tot=21 draw=15 invalid=0
cross=7 naughts=3 tot=31 draw=21 invalid=0



KeyboardInterrupt: 

100%|██████████| 100000/100000 [01:39<00:00, 1008.20it/s]


Стратегия иногда проигрывает, попробуем увеличить количество игр при построении дерева.

In [17]:
mtcs_agent = MCTSAgent(num_rounds=30_000)
random.seed(1000)
counter2 = TikTakCounter()
env = TicTacToe(4, 4, 3)
for i in tqdm.tqdm_notebook(range(10)):
    env.reset()
    play_rand_game(mtcs_agent, env=env, counter=counter2)
    print(counter2)

<ipython-input-17-a61cac3c8d1f>:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm.tqdm_notebook(range(10)):


cross=1 naughts=0 tot=1 draw=0 invalid=0
cross=2 naughts=0 tot=2 draw=0 invalid=0
cross=3 naughts=0 tot=3 draw=0 invalid=0
cross=3 naughts=1 tot=4 draw=0 invalid=0
cross=4 naughts=1 tot=5 draw=0 invalid=0
cross=5 naughts=1 tot=6 draw=0 invalid=0
cross=5 naughts=2 tot=7 draw=0 invalid=0
cross=6 naughts=2 tot=8 draw=0 invalid=0
cross=7 naughts=2 tot=9 draw=0 invalid=0
cross=8 naughts=2 tot=10 draw=0 invalid=0



Натренеруем Q-learning агента

In [22]:
env = TicTacToe(4, 4, 4)
agent_q = QleaningAgent(env)
counter2 = TikTakCounter(100)
N_GAME=100_000
for i in tqdm.tqdm(range(N_GAME)):
    agent_q.new_game(-1)
    play_game(env, None, agent_q, counter2,
                    True, False, verbose=False)
    counter2.tot += 1
    counter2.update_history()

100%|██████████| 100000/100000 [55:28<00:00, 30.05it/s]


In [23]:
print(counter2)

cross=3312 naughts=38544 tot=100000 draw=90 invalid=58054


Сыграем agent_q против mtcs_agent

In [31]:
def play_game2(agent1, agent2, env, counter: TikTakCounter):
    done = False

    while not done:
        action = agent1.get_action(env)
        _, reward, done, _ = env.step(action)
        if done:
            update_counter(counter, reward)
            break
        state = env.getHash()
        action = agent2.get_next_step(state, env, False)
        _, reward, done, _ = env.step(env.action_from_int(action))
        if done:
            update_counter(counter, reward)

In [33]:
env = TicTacToe(4, 4, 4)
mtcs_agent = MCTSAgent(num_rounds=10_000)
counter3 = TikTakCounter()
N_GAME=20
for i in tqdm.tqdm(range(N_GAME)):
    env.reset()
    agent_q.new_game(-1)
    play_game2(mtcs_agent, agent_q, env, counter3)
    counter3.tot += 1
    counter3.update_history()

100%|██████████| 20/20 [19:52<00:00, 59.63s/it]


In [34]:
print(counter3)

cross=0 naughts=10 tot=40 draw=0 invalid=10


q-learning выигрывает у mcts-tree, потому что в mcts мало вариантов опробования, из-за медленной реализации. Если бы она работала быстрее, то возможно было бы и получше.